# Algorithmic Trading with Deep Learning
This notebook demonstrates how to apply a Long Short-Term Memory (LSTM) model to predict stock prices based on historical data.

We will:
- Preprocess the data for the LSTM model
- Build and train an LSTM model
- Use the model to make predictions and visualize the results.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM


## Data Preparation
We will start by loading and preparing the data for training the LSTM model. This includes normalizing the data and creating sequences for the model to learn from.

In [ ]:

# Load your dataset
data = pd.read_csv('path_to_your_data.csv', index_col='DateTime', parse_dates=True)

# Feature selection: You can use 'Price' or a combination of 'Price' and other indicators you've computed.
dataset = data[['Price']].values

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Prepare the training dataset
sequence_length = 60  # Using the past 60 days to predict the next day
X_train, y_train = [], []

for i in range(sequence_length, len(scaled_data)):
    X_train.append(scaled_data[i-sequence_length:i, 0])
    y_train.append(scaled_data[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape the data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


## Building and Training the LSTM Model
Next, we will build an LSTM model and train it on the prepared data.

In [ ]:

# Initialize the LSTM model
model = Sequential()

# Adding the first LSTM layer with Dropout regularization
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=25))

# Adding a second LSTM layer
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))

# Output layer
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32)


## Making Predictions and Visualization
Finally, we will use the trained model to make predictions on the test data and visualize the predicted prices against the actual prices.

In [ ]:

# Create the test data set
test_data = scaled_data[-sequence_length:]
X_test = []

for i in range(sequence_length, len(test_data)):
    X_test.append(test_data[i-sequence_length:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Get the predicted prices
predicted_price = model.predict(X_test)
predicted_price = scaler.inverse_transform(predicted_price)

# Plot the predictions
plt.figure(figsize=(10, 6))
plt.plot(data.index[-len(predicted_price):], dataset[-len(predicted_price):], color='blue', label='Actual Price')
plt.plot(data.index[-len(predicted_price):], predicted_price, color='red', label='Predicted Price')
plt.title('Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()
